# Chess Engine with TensorFlow

## Dataset

In [8]:
import os

files = [file for file in os.listdir("E:\Smolfish\Smolfish\data") if file.endswith(".pgn")]

<>:3: SyntaxWarning: invalid escape sequence '\S'
<>:3: SyntaxWarning: invalid escape sequence '\S'
C:\Users\huynh\AppData\Local\Temp\ipykernel_11276\1612990722.py:3: SyntaxWarning: invalid escape sequence '\S'
  files = [file for file in os.listdir("E:\Smolfish\Smolfish\data") if file.endswith(".pgn")]


In [9]:
len(files)

79

In [11]:
!pip install chess
from chess import pgn

def load_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            game = pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    return games

     ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
     - -------------------------------------- 0.3/6.1 MB ? eta -:--:--
     --------------- ------------------------ 2.4/6.1 MB 9.6 MB/s eta 0:00:01
     --------------------------- ------------ 4.2/6.1 MB 9.0 MB/s eta 0:00:01
     ---------------------------------------  6.0/6.1 MB 9.5 MB/s eta 0:00:01
     ---------------------------------------- 6.1/6.1 MB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147814 sha256=b5633a3d2201a3d49f3a91d4a7e082bddaa30ed08d32ee991e074dcbb8dcb33a
  Stored in directory: c:\users\huynh\appdata\local\pip\cache\wheels\83\1f\4e\8f4300f7dd554eb8de70ddfed96e94d3d030ace10c5b53d447
Successfully built chess


In [13]:
from tqdm import tqdm

LIMIT_OF_FILES = min(len(files), 24)
games = []
i = 1
for file in tqdm(files):
    games.extend(load_pgn(f"E:\Smolfish\Smolfish\data\{file}"))
    if (i >= LIMIT_OF_FILES):
        break
    i += 1

<string>:7: SyntaxWarning: invalid escape sequence '\{'
<>:7: SyntaxWarning: invalid escape sequence '\{'
<>:7: SyntaxWarning: invalid escape sequence '\S'
<string>:7: SyntaxWarning: invalid escape sequence '\{'
<>:7: SyntaxWarning: invalid escape sequence '\{'
<>:7: SyntaxWarning: invalid escape sequence '\S'
C:\Users\huynh\AppData\Local\Temp\ipykernel_11276\2115550800.py:7: SyntaxWarning: invalid escape sequence '\{'
  games.extend(load_pgn(f"E:\Smolfish\Smolfish\data\{file}"))
C:\Users\huynh\AppData\Local\Temp\ipykernel_11276\2115550800.py:7: SyntaxWarning: invalid escape sequence '\S'
  games.extend(load_pgn(f"E:\Smolfish\Smolfish\data\{file}"))
 29%|██▉       | 23/79 [01:17<03:07,  3.35s/it]


In [14]:
len(games)

10327

## Build & train a neural network

In [16]:
!pip install tensorflow
import numpy as np
from chess import Board
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam # type: ignore
import time

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires protobuf<6,>=3.20, but you have protobuf 6.33.1 which is incompatible.


  Using cached tensorflow-2.20.0-cp312-cp312-win_amd64.whl.metadata (4.6 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached namex-0.1.0-py3-none-any.whl.metadata (322 bytes)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
Using cached tensorflow-2.20.0-cp312-cp312-win_amd64.whl (331.9 MB)
Using cached absl_py-2.3.1-py3-none-any.whl (135 kB)
Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
   ---------------------------------------- 0.0/4.7 MB ? eta -:--:--
   -------- ------------------------------- 1.0/4.7 M

In [17]:
def board_to_matrix(board: Board):
    matrix = np.zeros((8, 8, 12))
    piece_map = board.piece_map()
    for square, piece in piece_map.items():
        row, col = divmod(square, 8)
        piece_type = piece.piece_type - 1
        piece_color = 0 if piece.color else 6
        matrix[row, col, piece_type + piece_color] = 1
    return matrix


def create_input_for_nn(games):
    X = []
    y = []
    for game in games:
        board = game.board()
        for move in game.mainline_moves():
            X.append(board_to_matrix(board))
            y.append(move.uci())
            board.push(move)
    return X, y


def encode_moves(moves):
    move_to_int = {move: idx for idx, move in enumerate(set(moves))}
    return [move_to_int[move] for move in moves], move_to_int

In [19]:
X, y = create_input_for_nn(games)
y, move_to_int = encode_moves(y)
y = to_categorical(y, num_classes=len(move_to_int))
X = np.array(X, dtype=np.int8)

In [ ]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(8, 8, 12)),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(len(move_to_int), activation='softmax')
])
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(X, y, epochs=10, validation_split=0.1, batch_size=64)
model.save("models/TF_50EPOCHS.keras")

d:\Anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 6, 6, 64)       │         6,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1867)           │       479,819 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,085,195 (4.14 MB)

 Trainable params: 1,085,195 (4.14 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
 6925/11931 ━━━━━━━━━━━━━━━━━━━━ 1:53 23ms/step - accuracy: 0.0721 - loss: 5.8929

KeyboardInterrupt: 

## Predictions

In [ ]:
# Load the trained model
from tensorflow.keras.models import load_model
model = load_model("../../models/TF_50EPOCHS.keras")

In [ ]:
int_to_move = dict(zip(move_to_int.values(), move_to_int.keys()))


def predict_next_move(board):
    board_matrix = board_to_matrix(board).reshape(1, 8, 8, 12)
    predictions = model.predict(board_matrix)[0]
    legal_moves = list(board.legal_moves)
    legal_moves_uci = [move.uci() for move in legal_moves]
    sorted_indices = np.argsort(predictions)[::-1]
    for move_index in sorted_indices:
        move = int_to_move[move_index]
        if move in legal_moves_uci:
            return move
    return None

In [ ]:
# Create a chess board (start position)
board = Board()

In [ ]:
# Display the board before prediction
print("Board before prediction:")
board

# Predict and make the move
next_move = predict_next_move(board)
board.push_uci(next_move)

# Display the board after prediction
print("\nPredicted move:", next_move)
print("Board after prediction:")
board

In [ ]:
print(str(pgn.Game.from_board(board)))